In [1]:
#Install the nessasary packages from pip
!pip install mediapipe opencv-python pandas scikit-learn --user

In [1]:
#Import all the nessacary libraries from the packages

#Import mediapipe to be use as the model
import mediapipe as mp
#Import opencv for rendaring and drawing capabilities
import cv2

import numpy as np #Handle numpy arrays
import pandas as pd #Handle tabular data
import csv #Handle csv files
import os #Handle folder structure
import glob
import pickle #Save and oad ML model

from sklearn.model_selection import train_test_split #Partition the data into training and testing

from sklearn.pipeline import make_pipeline #Creates a pipeline
from sklearn.preprocessing import StandardScaler #Standadize data 

#Classification algorithms
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score #Evaluate model through accuracy

In [2]:
#Helper to draw the landmarks and provide the landmark detection models
draw_helpers = mp.solutions.drawing_utils 
holistic_model = mp.solutions.holistic 

In [4]:
#DETECT AND SAVE THE LANDMARKS COORDS AS COLUMNS FROM THE SAMPLE VIDEO

#Connect the sample video from the device
sample_video = cv2.VideoCapture('1.valid.mp4')

#Load the holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through each frame of the video 
    while sample_video.isOpened():
        #Returns the status of the read and the frame as an image
        status, frame = sample_video.read()
        
        #If frame is read correctly, status is true
        if status == False:
            print("Exit!")
            break
          
        #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Prevent writing and copying frame data to improve performance while making the detection
        rgb_frame.flags.writeable = False        
        
        #Use holistic model to make detections
        result_frame = holistic.process(rgb_frame)
        
        #Set frame back to writable format after detection
        rgb_frame.flags.writeable = True   
        
        #Recolor the captured frame from BGR for rendering with opencv
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        #Use pose model to detect only the landmarks of the the body and not the landmarks of the face and hand
        draw_helpers.draw_landmarks(bgr_frame, result_frame.pose_landmarks, holistic_model.POSE_CONNECTIONS, 
                                 draw_helpers.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 draw_helpers.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
         
        #Display the frames    
        cv2.imshow('Raw Feed', bgr_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
sample_video.release()
cv2.destroyAllWindows()

Cannot receive frame!


In [5]:
#Store the pose landmarks in an array 
landmarks_array = result_frame.pose_landmarks.landmark

#Save the no of pose landmarks in the Array 
num_landmarks = len(landmarks_array)
#Display landmarks
num_landmarks

33

In [6]:
#Save the landmarks to a table to be exported as a csv file

#0th column of the table
table_columns = ['class']
#Add columns to the table according to the no.of landmarks
for num in range(1, num_landmarks + 1):
    table_columns += ['x{}'.format(num), 'y{}'.format(num), 'z{}'.format(num), 'v{}'.format(num)]
    
#Display columns of the table
table_columns

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [7]:
#Write to the csv file
with open('data.csv', mode='w', newline='') as f:
    #Define the csv writer
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(table_columns)

In [24]:
#DETECT AND SAVE COORDS OF LANDMARKS FOR EACH CLASS THROUGH THE TRAIN VIDEO

def video_processor(class_name, video_pth):
    #Connect the sample video from the device
    sample_video = cv2.VideoCapture(video_pth)

    #Load the holistic model
    with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        #Loop through each frame of the video 
        while sample_video.isOpened():
            #Returns the status of the read and the frame as an image
            status, frame = sample_video.read()

            #If frame is read correctly, status is true
            if status == False:
                print("Exit!")
                break

            #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            #Prevent writing and copying frame data to improve performance while making the detection
            rgb_frame.flags.writeable = False        

            #Use holistic model to make detections
            result_frame = holistic.process(rgb_frame)

            #Set frame back to writable format after detection
            rgb_frame.flags.writeable = True   

            #Recolor the captured frame from BGR for rendering with opencv
            bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

            #Use pose model to detect only the landmarks of the the body and not the landmarks of the face and hand
            draw_helpers.draw_landmarks(bgr_frame, result_frame.pose_landmarks, holistic_model.POSE_CONNECTIONS, 
                                     draw_helpers.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                     draw_helpers.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )
         
            #Export the coordinates of the landmarks to the csv file
            try:

                #Extracting all the landmarks of the pose as an array
                pose_landmarks_array = result_frame.pose_landmarks.landmark
                #Format landmarks in to a numpy array for better structuring(removing keys) and collapse array to 1 dimesnsion
                pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks_array]).flatten() 
                                              if result_frame.pose_landmarks else np.zeros(33*4))

                #Append class name as the Oth element
                pose_landmarks_nparray.insert(0, class_name)
                
                #Append the data to table in the csv file
                with open('data.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(pose_landmarks_nparray) 
            except:
                pass
            
            #Display the frames    
            cv2.imshow('Raw Feed', bgr_frame)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    sample_video.release()
    cv2.destroyAllWindows()

In [25]:
video_processor("Correct", "3.valid.mp4")

['Correct', 0.5347889065742493, 0.34001779556274414, -0.7344223856925964, 0.9999992847442627, 0.5450612902641296, 0.32787302136421204, -0.6882516145706177, 0.9999988079071045, 0.5523107051849365, 0.32788240909576416, -0.6881699562072754, 0.9999982118606567, 0.5590097308158875, 0.3280247747898102, -0.6883479356765747, 0.9999984502792358, 0.5212579965591431, 0.3273196816444397, -0.6901722550392151, 0.9999986886978149, 0.5106821656227112, 0.3270440101623535, -0.6902960538864136, 0.9999982118606567, 0.5012689232826233, 0.3269404172897339, -0.6902133226394653, 0.9999988079071045, 0.5652154088020325, 0.3305651843547821, -0.4238899350166321, 0.9999982118606567, 0.48601627349853516, 0.32957908511161804, -0.4324539601802826, 0.9999991655349731, 0.5452933311462402, 0.3514213562011719, -0.635833203792572, 0.9999995231628418, 0.5151342749595642, 0.35119685530662537, -0.6388363242149353, 0.9999996423721313, 0.6211651563644409, 0.3840508759021759, -0.268207311630249, 0.9999827146530151, 0.4182841479

['Correct', 0.5351625680923462, 0.34038880467414856, -0.6600316762924194, 0.9999991059303284, 0.5448428392410278, 0.32829800248146057, -0.6160615086555481, 0.9999986290931702, 0.5520414710044861, 0.32817384600639343, -0.6159844994544983, 0.9999979734420776, 0.5584955215454102, 0.3282094895839691, -0.6161626577377319, 0.9999982118606567, 0.521734356880188, 0.32803773880004883, -0.6178780198097229, 0.9999985098838806, 0.5113550424575806, 0.3277130126953125, -0.6179739236831665, 0.9999979734420776, 0.5019165873527527, 0.3275599777698517, -0.617866575717926, 0.9999986886978149, 0.5659788846969604, 0.3307228982448578, -0.36000826954841614, 0.9999979734420776, 0.48554179072380066, 0.3299888074398041, -0.36753854155540466, 0.9999990463256836, 0.5457565188407898, 0.3518851697444916, -0.5645251870155334, 0.9999994039535522, 0.5180187225341797, 0.35227519273757935, -0.5670415759086609, 0.9999996423721313, 0.6220930218696594, 0.38540422916412354, -0.20690003037452698, 0.9999788403511047, 0.418173

['Correct', 0.531981348991394, 0.3411286473274231, -0.7291802763938904, 0.9999989867210388, 0.5412123799324036, 0.3290858864784241, -0.6855315566062927, 0.9999986290931702, 0.5484874248504639, 0.32884737849235535, -0.6854215860366821, 0.9999979138374329, 0.5555481314659119, 0.3287763297557831, -0.6855971217155457, 0.999998152256012, 0.5172995328903198, 0.328811377286911, -0.68618243932724, 0.9999984502792358, 0.5074922442436218, 0.3284459412097931, -0.6862970590591431, 0.9999978542327881, 0.49864819645881653, 0.3282308280467987, -0.6861838102340698, 0.9999986290931702, 0.564271867275238, 0.33105504512786865, -0.4264044165611267, 0.9999979138374329, 0.4834171533584595, 0.3305000066757202, -0.42753612995147705, 0.9999989867210388, 0.5435933470726013, 0.35266250371932983, -0.6319559812545776, 0.9999992847442627, 0.513282299041748, 0.35334667563438416, -0.6328424215316772, 0.9999996423721313, 0.6226540803909302, 0.3839121460914612, -0.26956653594970703, 0.9999765157699585, 0.41798189282417

['Correct', 0.5302431583404541, 0.3405337333679199, -0.5938014984130859, 0.9999987483024597, 0.5391584038734436, 0.32889795303344727, -0.5505770444869995, 0.9999982714653015, 0.5458822846412659, 0.3286794424057007, -0.5504336953163147, 0.9999973177909851, 0.5528301000595093, 0.32857340574264526, -0.5505731105804443, 0.9999976754188538, 0.5157289505004883, 0.32858699560165405, -0.5520604252815247, 0.9999980926513672, 0.5060840845108032, 0.32826918363571167, -0.5521658062934875, 0.9999972581863403, 0.49711015820503235, 0.3281184434890747, -0.5520576238632202, 0.9999983310699463, 0.5604882836341858, 0.33067211508750916, -0.30983611941337585, 0.9999970197677612, 0.4803870916366577, 0.33019426465034485, -0.3137310743331909, 0.9999988675117493, 0.5417619943618774, 0.3519826829433441, -0.5041257739067078, 0.9999988079071045, 0.5116393566131592, 0.3529469668865204, -0.505820095539093, 0.9999994039535522, 0.6195801496505737, 0.3750665783882141, -0.1616978645324707, 0.9999692440032959, 0.4164442

['Correct', 0.5227162837982178, 0.3393200933933258, -0.6799502372741699, 0.9999954700469971, 0.5312623977661133, 0.32721689343452454, -0.6392609477043152, 0.9999942779541016, 0.5382245182991028, 0.3269001841545105, -0.6391026377677917, 0.9999918341636658, 0.5445783734321594, 0.326616108417511, -0.6392993330955505, 0.9999936819076538, 0.5087791681289673, 0.32790133357048035, -0.634563148021698, 0.9999927878379822, 0.5006979703903198, 0.3282478451728821, -0.6347313523292542, 0.9999890923500061, 0.4929855763912201, 0.3287002146244049, -0.6346263885498047, 0.9999926090240479, 0.5528323650360107, 0.32948410511016846, -0.42704150080680847, 0.9999901056289673, 0.47800546884536743, 0.3310282528400421, -0.4025671184062958, 0.999991774559021, 0.5369373559951782, 0.35067421197891235, -0.6009308695793152, 0.9999942779541016, 0.5082293152809143, 0.35217565298080444, -0.5939439535140991, 0.999994695186615, 0.6162183880805969, 0.3697768449783325, -0.3887633979320526, 0.99993896484375, 0.4173924624919

...

['Correct', 0.5157321095466614, 0.3404863178730011, -0.5841096043586731, 0.9999682307243347, 0.5255340337753296, 0.32705286145210266, -0.5383631587028503, 0.9999634623527527, 0.5330604910850525, 0.32611849904060364, -0.5384016633033752, 0.9999579191207886, 0.5408629775047302, 0.32539448142051697, -0.538615345954895, 0.9999648928642273, 0.5020701885223389, 0.32893186807632446, -0.5300118327140808, 0.9999420046806335, 0.49449214339256287, 0.3294028341770172, -0.5303303599357605, 0.9999251961708069, 0.48702239990234375, 0.32997217774391174, -0.5305026173591614, 0.9999321699142456, 0.5495577454566956, 0.32846599817276, -0.3428458869457245, 0.9999490976333618, 0.4766375422477722, 0.33232980966567993, -0.3108113408088684, 0.9999142289161682, 0.5318464636802673, 0.35109883546829224, -0.5145933628082275, 0.9999493360519409, 0.5032893419265747, 0.3535023629665375, -0.5052717924118042, 0.9999272227287292, 0.6174126267433167, 0.37068048119544983, -0.2989673614501953, 0.9998403191566467, 0.4205430

['Correct', 0.5204286575317383, 0.3397528827190399, -0.569636344909668, 0.9999799132347107, 0.5293983817100525, 0.32672423124313354, -0.5267228484153748, 0.999975860118866, 0.5365104675292969, 0.32614192366600037, -0.5265855193138123, 0.9999714493751526, 0.5431370735168457, 0.32564985752105713, -0.5267847180366516, 0.9999764561653137, 0.5059165954589844, 0.3278340995311737, -0.5269516706466675, 0.9999632239341736, 0.49765637516975403, 0.3281114399433136, -0.5270873308181763, 0.9999517798423767, 0.4897308945655823, 0.3284659683704376, -0.5269534587860107, 0.9999582171440125, 0.5507408380508423, 0.32827338576316833, -0.3081827461719513, 0.9999651312828064, 0.47655317187309265, 0.3310078978538513, -0.30663925409317017, 0.9999485015869141, 0.5347198247909546, 0.3509005904197693, -0.489793062210083, 0.9999693632125854, 0.505287230014801, 0.35241127014160156, -0.4892147481441498, 0.9999576210975647, 0.6202284693717957, 0.37405064702033997, -0.22195661067962646, 0.9998959302902222, 0.41560402

['Correct', 0.5254546403884888, 0.34085017442703247, -0.5997089147567749, 0.9999874234199524, 0.5343775153160095, 0.328798770904541, -0.5558165907859802, 0.9999842047691345, 0.5403807759284973, 0.32853999733924866, -0.5556918382644653, 0.9999807476997375, 0.547240674495697, 0.3281911313533783, -0.5558467507362366, 0.9999840259552002, 0.5108754634857178, 0.32888805866241455, -0.558520495891571, 0.9999771118164062, 0.5016536712646484, 0.3288908898830414, -0.5586132407188416, 0.9999696016311646, 0.49330493807792664, 0.32898515462875366, -0.5584947466850281, 0.9999747276306152, 0.5560522079467773, 0.3296574354171753, -0.32056066393852234, 0.9999762177467346, 0.47737234830856323, 0.3304729163646698, -0.33210986852645874, 0.9999701380729675, 0.5376736521720886, 0.35221627354621887, -0.5130554437637329, 0.999981701374054, 0.5076489448547363, 0.3540622293949127, -0.5166153907775879, 0.9999760389328003, 0.6203588843345642, 0.37698617577552795, -0.20802010595798492, 0.9999276399612427, 0.4154805

['Correct', 0.5270311832427979, 0.34108418226242065, -0.6789262294769287, 0.9999927878379822, 0.5357174873352051, 0.32911014556884766, -0.6336621046066284, 0.999990701675415, 0.5415494441986084, 0.32881495356559753, -0.6335341930389404, 0.9999885559082031, 0.5487592220306396, 0.32847338914871216, -0.6336888074874878, 0.9999903440475464, 0.5124704241752625, 0.32924583554267883, -0.6348159909248352, 0.9999868273735046, 0.5034496784210205, 0.3292083442211151, -0.6349349617958069, 0.9999825954437256, 0.4946286380290985, 0.3293091952800751, -0.6348137259483337, 0.9999857544898987, 0.5570091605186462, 0.3301820158958435, -0.38262149691581726, 0.9999858736991882, 0.47761908173561096, 0.3308170735836029, -0.3882015347480774, 0.999983549118042, 0.5383214354515076, 0.35262906551361084, -0.5861324071884155, 0.9999898076057434, 0.5083678364753723, 0.3543800711631775, -0.587907612323761, 0.9999869465827942, 0.6196280121803284, 0.3796209692955017, -0.24533049762248993, 0.9999499320983887, 0.41218408

['Correct', 0.5272730588912964, 0.34182992577552795, -0.691383957862854, 0.9999955296516418, 0.5358774662017822, 0.329459547996521, -0.6477742195129395, 0.9999940991401672, 0.5418325662612915, 0.3292035162448883, -0.6476388573646545, 0.9999925494194031, 0.5492407083511353, 0.3289564847946167, -0.6478009819984436, 0.9999936819076538, 0.5127344727516174, 0.32953280210494995, -0.648964524269104, 0.9999919533729553, 0.5033899545669556, 0.32942667603492737, -0.6490887403488159, 0.9999893307685852, 0.49462273716926575, 0.32946833968162537, -0.6489685773849487, 0.9999915361404419, 0.5577661395072937, 0.33051547408103943, -0.4017729163169861, 0.9999909996986389, 0.47784820199012756, 0.3310050070285797, -0.40639135241508484, 0.9999906420707703, 0.5383223295211792, 0.35360005497932434, -0.6001875996589661, 0.999994158744812, 0.5084757208824158, 0.3550568222999573, -0.602509081363678, 0.9999927878379822, 0.6193907856941223, 0.381517618894577, -0.245563343167305, 0.9999614357948303, 0.412013798952

In [120]:
#Specify the location to the datasets
valid_path = "./datasets/valid/*.mp4"
invalid_path = "./datasets/invalid/*.mp4"

#Displat the no of files in the datasets
print("Valid Video Count: ", len(glob.glob(valid_path)))
print("Invalid Video Count: ", len(glob.glob(invalid_path)))

Valid video count :  30
Invalid video count : 11


In [ ]:
#Adding landmarks of the invalid dataset to the csv
class_name = "Correct"
for i in range (1, len(glob.glob(valid_path)) + 1):
    video_pth = "./datasets/valid/" + str(i) + ".mp4"
    video_processor(class_name, video_pth)
    print("Processed: ", str(i)

In [ ]:
#Adding landmarks of the invalid dataset to the csv
class_name = "Incorrect"
for i in range (1, len(glob.glob(invalid_path)) + 1):
    video_pth = "./datasets/invalid/" + str(i) + ".mp4"
    video_processor(class_name, video_pth)
    print("Processed: ", str(i)

In [26]:
#Import dataframe
df = pd.read_csv('data.csv')

In [27]:
#Display the first 5 rows in the dataframe
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Correct,0.534789,0.340018,-0.734422,0.999999,0.545061,0.327873,-0.688252,0.999999,0.552311,...,-0.472349,0.937949,0.645837,0.759634,-0.685680,0.975963,0.406702,0.761222,-0.760625,0.981579
1,Correct,0.534788,0.340075,-0.679886,0.999999,0.544802,0.327912,-0.635636,0.999999,0.552053,...,-0.587094,0.938885,0.649047,0.759775,-0.824108,0.976163,0.405543,0.761442,-0.881457,0.981426
2,Correct,0.534851,0.340163,-0.643253,0.999999,0.544784,0.328017,-0.599718,0.999999,0.552030,...,-0.594323,0.939786,0.650265,0.759939,-0.822347,0.976234,0.404899,0.761539,-0.884187,0.981182
3,Correct,0.534901,0.340234,-0.639145,0.999999,0.544768,0.328118,-0.595136,0.999999,0.552000,...,-0.601119,0.940453,0.650691,0.760146,-0.839792,0.976385,0.404781,0.761661,-0.892310,0.980968
4,Correct,0.535042,0.340296,-0.648428,0.999999,0.544791,0.328195,-0.604258,0.999999,0.552007,...,-0.596905,0.940832,0.650841,0.760248,-0.832493,0.976366,0.404780,0.761688,-0.888286,0.980666


In [28]:
#Display the last 5 rows in the dataframe
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
76,Correct,0.527193,0.341830,-0.696658,0.999995,0.535855,0.329449,-0.653082,0.999994,0.541785,...,-0.477941,0.931223,0.649873,0.760314,-0.699613,0.974437,0.405546,0.762178,-0.757666,0.977532
77,Correct,0.527273,0.341830,-0.691384,0.999996,0.535877,0.329460,-0.647774,0.999994,0.541833,...,-0.490456,0.931123,0.649910,0.760346,-0.720183,0.974255,0.405584,0.762178,-0.770985,0.977179
78,Correct,0.527475,0.341830,-0.682671,0.999996,0.535949,0.329476,-0.640857,0.999994,0.541972,...,-0.505001,0.930721,0.650216,0.760388,-0.744007,0.974045,0.405910,0.762099,-0.785265,0.976663
79,Correct,0.527481,0.341830,-0.668001,0.999996,0.535951,0.329490,-0.627740,0.999995,0.541982,...,-0.495164,0.930214,0.650356,0.760399,-0.727630,0.973676,0.406083,0.762012,-0.772083,0.976117
80,Correct,0.527493,0.341825,-0.668258,0.999996,0.535957,0.329493,-0.627790,0.999995,0.542005,...,-0.492280,0.929735,0.650851,0.760377,-0.719618,0.973319,0.406098,0.761930,-0.766681,0.975687


In [30]:
#Remove the class column so the dataframe only contains features
X = df.drop('class', axis=1)
#Use the class as the target value
Y = df['class'] 

0     Correct
1     Correct
2     Correct
3     Correct
4     Correct
       ...   
76    Correct
77    Correct
78    Correct
79    Correct
80    Correct
Name: class, Length: 81, dtype: object

In [31]:
#Split the data with 30% for testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1234)

In [32]:
#Setup the machine learning model pipeline
model_pipeline = make_pipeline(StandardScaler(), RandomForestClassifier())

In [64]:
#Train the model
model = model_pipeline.fit(X_train.values, Y_train.values)

In [65]:
#Predict the class of the test data
output_class = model.predict(X_test.values)

#Comparing the output labels with the test data labels
print("Accuracy of the model: ", accuracy_score(Y_test, output_class))

Accuracy of the model:  1.0


In [35]:
#Save the model as a binary file
with open('rfc_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [36]:
#Import the model from the binary file
with open('rfc_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [58]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [80]:
#PREDICT AND DISPLAY THE RESULTS OF THE MODEL BY PASSING THE TEST VIDEO

#Connect the test video from the device
sample_video = cv2.VideoCapture('5.invalid.mp4')

#Load the holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #Loop through each frame of the video 
    while sample_video.isOpened():
        #Returns the status of the read and the frame as an image
        status, frame = sample_video.read()
        
        #If frame is read correctly, status is true
        if status == False:
            print("Exit!")
            break
          
        #Recolor the captured frame from BGR to RGB (Medipipe requies frames to be in RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Prevent writing and copying frame data to improve performance while making the detection
        rgb_frame.flags.writeable = False        
        
        #Use holistic model to make detections
        result_frame = holistic.process(rgb_frame)
        
        #Set frame back to writable format after detection
        rgb_frame.flags.writeable = True   
        
        #Recolor the captured frame from BGR for rendering with opencv
        bgr_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

        #Use pose model to detect only the landmarks of the the body and not the landmarks of the face and hand
        draw_helpers.draw_landmarks(bgr_frame, result_frame.pose_landmarks, holistic_model.POSE_CONNECTIONS, 
                                 draw_helpers.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 draw_helpers.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
         #Predict the coordinates of the landmarks (resulrs screen)
        try:
            #Extracting all the landmarks of the pose as an array
            pose_landmarks_array = result_frame.pose_landmarks.landmark
            #Format landmarks in to a numpy array for better structuring(removing keys) and collapse array to 1 dimesnsion
            pose_landmarks_nparray = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks_array]).flatten() 
                                          if result_frame.pose_landmarks else np.zeros(33*4))
            
            #Pass the numpy array into a data frame
            features = pd.DataFrame([pose_landmarks_nparray])

            #Store the top class of the prediction
            pose_class_status = model.predict(features.values)[0]
            #Store the probability of the prediction
            pose_class_status_prob = model.predict_proba(features.values)[0]
            
            
            #Set a rectangle box to display the results of the prediction in the video frame
            #rectangle(container, top_coord, bottom_coord, color, line_thickness)
            cv2.rectangle(bgr_frame, (0,0), (250, 60), (245, 117, 16), -1)
            
            #Display the class label inside the rectangle box
            cv2.putText(bgr_frame, 'Class'
                        , (105,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract =and display the top class of the prediction
            cv2.putText(bgr_frame, pose_class_status.split(' ')[0]
                        , (100,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display the class probability inside the rectangle box
            cv2.putText(bgr_frame, 'Probability'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            #Extract and dispthe maximum probability
            cv2.putText(bgr_frame, str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
 
        except:
            pass
                        
        #Display the frames  

        cv2.imshow('Results Feed', bgr_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    print("Class:", pose_class_status, " Probability:", str(round(pose_class_status_prob[np.argmax(pose_class_status_prob)],2)))


sample_video.release()
cv2.destroyAllWindows()

Exit!
Class: Correct  Probability: 1.0
